In [1]:
# 1) (Optional) Mount Google Drive to persist checkpoints
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception as e:
    print('Not running in Colab or drive mount failed:', e)

Mounted at /content/drive


In [6]:
# 2) Clone the repository (or pull if already present)
import os, subprocess, sys

repo_name = '270FT'
# The notebook typically starts in /content in Colab.
# Define the absolute path to the intended repository location.
intended_repo_parent_path = '/content'
repo_absolute_path = os.path.join(intended_repo_parent_path, repo_name)

# Ensure we are in the intended parent directory before any git operations
# This prevents cloning into nested directories if CWD was changed previously
if os.getcwd() != intended_repo_parent_path:
    print(f"Changing directory from {os.getcwd()} to {intended_repo_parent_path}")
    os.chdir(intended_repo_parent_path)

if not os.path.exists(repo_absolute_path):
    print(f'Cloning repository into {repo_absolute_path}')
    # Clone directly into the target directory (will create repo_name inside CWD, which is now /content)
    subprocess.check_call(['git', 'clone', 'https://github.com/oleeveeuh/270FT.git', repo_name])
else:
    print(f'Repository already exists at {repo_absolute_path}; fetching latest changes')
    try:
        # Use -C with the absolute path to ensure git pull operates on the correct directory
        subprocess.check_call(['git', '-C', repo_absolute_path, 'pull'])
    except Exception as e:
        print('git pull failed:', e)

# Ensure the working directory is always the absolute path of the repository root
# This ensures that subsequent runs of the cell always land in the correct CWD
# and prevents issues with relative chdir calls.
if os.getcwd() != repo_absolute_path:
    print(f"Changing directory from {os.getcwd()} to {repo_absolute_path}")
    os.chdir(repo_absolute_path)

print('CWD:', os.getcwd())

Changing directory from /content/270FT to /content
Repository already exists at /content/270FT; fetching latest changes
Changing directory from /content to /content/270FT
CWD: /content/270FT


In [3]:
# 3) Install required Python packages (may take a few minutes).
# Install core packages first, then attempt bitsandbytes which may need a matching CUDA runtime.
!pip install -q transformers datasets peft evaluate pyyaml huggingface_hub wandb
# Try to install bitsandbytes; if it fails, you can retry with a wheel matching the runtime's CUDA
try:
    get_ipython().system('pip install -q bitsandbytes')
    print('bitsandbytes installed')
except Exception as e:
    print('bitsandbytes install failed (you may still proceed if using a different runtime):', e)
# Show GPU info if available
!nvidia-smi || true

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 46.1 MB/s eta 0:00:00
bitsandbytes installed
Fri Nov 28 05:31:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             44W /  400W |       0M

If `bitsandbytes` or other installs fail due to CUDA mismatches, try switching the Colab runtime GPU type (or use a notebook with a supported CUDA version). You can also skip installing `bitsandbytes` if you only want CPU runs, but the full QLoRA workflow requires a CUDA GPU.

In [4]:
# 4) (Optional) Preprocess raw data into processed JSONL files.
# This will create/update data/processed/train.jsonl, validation.jsonl, and test.jsonl
!python preprocess/load_and_prepare.py --raw_dir data/raw --processed_dir data/processed --validation_split 0.15 --test_split 0.15

Install with: pip install pdfplumber
Loading tokenizer: gpt2
tokenizer_config.json: 100% 26.0/26.0 [00:00<00:00, 186kB/s]
config.json: 100% 665/665 [00:00<00:00, 6.06MB/s]
vocab.json: 100% 1.04M/1.04M [00:00<00:00, 20.5MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 65.3MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 24.7MB/s]
Scanning for files in: /content/270FT/data/raw
No subdirectories detected. Using automatic splitting based on filename patterns...
Found 0 training files, 0 validation files, and 0 test files

Processing training files...

Processing validation files...

Processing test files...

Saving processed data to: /content/270FT/data/processed

[OK] Processed 0 training examples
[OK] Processed 0 test examples
[OK] Saved to /content/270FT/data/processed/train.jsonl
[OK] Saved to /content/270FT/data/processed/test.jsonl


In [4]:
# 5) (Optional) Inspect processed files sizes and line counts
!ls -lh data/processed || true
!wc -l data/processed/*.jsonl || true

total 264K
-rw-r--r-- 1 root root  11K Nov 28 05:32 test.jsonl
-rw-r--r-- 1 root root 207K Nov 28 05:32 train.jsonl
-rw-r--r-- 1 root root  43K Nov 28 05:32 validation.jsonl
     9 data/processed/test.jsonl
   111 data/processed/train.jsonl
    16 data/processed/validation.jsonl
   136 total


In [7]:
# 6) Provide Hugging Face token (if required).
# Use this cell to securely input a token if you need to access gated models.
from getpass import getpass
token = getpass('Hugging Face token (leave blank if not needed): ')
import os
if token:
    os.environ['HF_TOKEN'] = token
    # Also login via huggingface-cli for convenience
    try:
        get_ipython().system('huggingface-cli login --token "$HF_TOKEN"')
    except Exception as e:
        print('Automatic huggingface-cli login failed; token stored in HF_TOKEN')
    print('HF_TOKEN set in environment')
else:
    print('No token provided; proceeding without HF token')

Hugging Face token (leave blank if not needed): ··········
⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `270ft` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
HF_TOKEN set in environment


In [5]:
# 6b) Disable W&B interactive logging to avoid login prompts during automated runs
import os
os.environ['WANDB_MODE'] = 'offline'
print('W&B offline mode set')

W&B offline mode set


Now run the full training script below. The script `training/train_dual_lora.py` will look for `data/processed/train.jsonl`, `validation.jsonl`, and `test.jsonl` in `data/processed` and will use the models listed in `configs/training_config.yaml`.

**Note about test data**: The test set is reserved for human-in-the-loop evaluation after training. During training, only the validation set is used for evaluation metrics. The training will log:
- **Training loss** every 10 steps
- **Validation loss** at the end of each epoch
- Metrics to W&B (if enabled) or console output

In [7]:
# 7) Run the training script (this will start QLoRA fine-tuning).
# Note: training will log to W&B if enabled in the config; we set W&B to offline above to avoid interactive prompts.
# Use unbuffered output so logs stream in Colab
!python -u training/train_dual_lora.py

2025-11-28 06:31:51.153634: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-28 06:31:51.172690: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764311511.194849   20251 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764311511.201477   20251 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764311511.218852   20251 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

**Training Monitoring:**
The training script logs losses and metrics in multiple ways:
1. **Console output**: Training loss logged every 10 steps, validation metrics at each epoch
2. **W&B (if enabled)**: Full training/validation curves, model checkpoints
3. **Checkpoints**: Saved every 500 steps to the output directory (keeping last 3)

**Troubleshooting tips:**
- If you see `FileNotFoundError` complaining about missing `test.jsonl`, re-run the preprocessing cell or ensure `data/processed/test.jsonl` exists.
- If you get `bitsandbytes` import errors, try installing a different `bitsandbytes` wheel that matches the Colab CUDA runtime or switch runtime.
- If training runs out of VRAM, reduce `batch_size` in `configs/training_config.yaml` or switch to a larger GPU.
- If you prefer to persist checkpoints to Google Drive, create a folder in Drive and update `configs/training_config.yaml` output paths to point inside `/content/drive/MyDrive/...`.

**Next steps after training:**
- Run inference on test questions using the trained adapter
- Use human-in-the-loop review to evaluate the quality of generated solutions
- The test set at `data/processed/test.jsonl` contains questions without solutions (intentional for unbiased evaluation)

## Understanding Your Results

### Automated Metrics (for questions with reference solutions)
- **Exact Match Rate**: Percentage of solutions that exactly match your reference (strict comparison)
- **BLEU Score**: Token-level similarity score (0-1 scale)
  - 0.0-0.1: Poor match
  - 0.1-0.3: Fair match  
  - 0.3-0.5: Good match
  - 0.5+: Excellent match

### Quality Checks (for all questions)
All generated solutions are automatically checked for:
- ✓/✗ Has algorithm/pseudocode section
- ✓/✗ Has runtime analysis (Big-O notation)
- ✓/✗ Has proof keywords (proof, correctness, invariant, etc.)
- ✓/✗ Has code structure (for/while/if statements)
- Detected complexity notations
- Length validation

### Human Review Workflow
For questions without reference solutions:
1. Download the `human_review_*.csv` file (cell above)
2. Open in Excel or Google Sheets
3. Review each generated solution
4. Fill in the "Rating (1-5)" column:
   - 1 = Incorrect/useless
   - 2 = Major issues
   - 3 = Acceptable but flawed
   - 4 = Good with minor issues
   - 5 = Excellent
5. Add comments explaining your rating
6. Calculate average rating and % of items rated 4-5

For complete documentation, see [EVALUATION_GUIDE.md](../EVALUATION_GUIDE.md) and [EVALUATION_QUICKSTART.md](../EVALUATION_QUICKSTART.md)

In [ ]:
# 11) Download results for human review
# This cell creates downloadable files:
# - evaluation_*.json: Full automated metrics and quality checks
# - human_review_*.csv: Template for manual review of items without solutions

from google.colab import files
from pathlib import Path
import os

results_dir = Path('results')
if results_dir.exists():
    # Download JSON results
    json_files = list(results_dir.glob('evaluation_*.json'))
    if json_files:
        latest_json = max(json_files, key=os.path.getmtime)
        print(f"Downloading: {latest_json.name}")
        files.download(str(latest_json))

    # Download CSV for human review
    csv_files = list(results_dir.glob('human_review_*.csv'))
    if csv_files:
        latest_csv = max(csv_files, key=os.path.getmtime)
        print(f"Downloading: {latest_csv.name}")
        files.download(str(latest_csv))
        print("\\n✓ Open the CSV in Excel or Google Sheets to complete human review")
        print("  Fill in 'Rating (1-5)' and 'Comments' columns")
    else:
        print("No CSV for human review (all items have reference solutions)")
else:
    print("Results directory not found. Run evaluation first.")

In [ ]:
# 10) View evaluation results summary
import json
from pathlib import Path
import os

results_dir = Path('results')
if results_dir.exists():
    # Find the most recent evaluation JSON
    json_files = list(results_dir.glob('evaluation_*.json'))
    if json_files:
        latest_json = max(json_files, key=os.path.getmtime)

        with open(latest_json, 'r') as f:
            results = json.load(f)

        print(f"{'='*60}")
        print(f"EVALUATION RESULTS: {results['model_name']}")
        print(f"{'='*60}\\n")

        print(f"Total Test Items: {results['total_items']}")
        print(f"  Items with reference solutions: {results['items_with_solutions']}")
        print(f"  Items needing human review: {results['items_without_solutions']}\\n")

        if results['items_with_solutions'] > 0:
            print(f"AUTOMATED METRICS (on {results['items_with_solutions']} items with solutions):")
            print(f"  Exact Match Rate: {results['automated_metrics']['exact_match_rate']:.4f} ({results['automated_metrics']['exact_matches']}/{results['items_with_solutions']} matched)")
            print(f"  Average BLEU Score: {results['automated_metrics']['avg_bleu_score']:.4f}")
            print(f"    (0.0-0.1: Poor, 0.1-0.3: Fair, 0.3-0.5: Good, 0.5+: Excellent)\\n")

        if results['items_without_solutions'] > 0:
            print(f"HUMAN REVIEW NEEDED ({results['items_without_solutions']} items):")
            print(f"  CSV template exported for manual review")
            print(f"  Quality pre-checks completed\\n")

        print(f"\\nDetailed results saved to: {latest_json}")
    else:
        print("No evaluation results found. Run evaluation first.")
else:
    print("Results directory not found. Run evaluation first.")

In [ ]:
# 9) Run hybrid evaluation (automated metrics + quality checks)
# This will:
# - Generate solutions for all test questions using your fine-tuned model
# - Run automated metrics (BLEU, exact match) for questions with reference solutions
# - Run quality checks (structure, completeness) for all questions
# - Flag questions without solutions for human review
# - Export results to JSON and CSV

!python -u evaluation/evaluate_with_solutions.py

In [ ]:
# 8b) Run the reference solutions script
# Only run this after editing REFERENCE_SOLUTIONS in the cell above
!python temp_add_solutions.py

In [ ]:
# 8) Add reference solutions for test questions (where you have them)
# Edit the REFERENCE_SOLUTIONS dictionary below to add your solutions

reference_solutions_script = """
import json
from pathlib import Path

# ADD YOUR REFERENCE SOLUTIONS HERE
# Format: question ID (0-indexed) -> reference solution
# Leave empty string "" for questions without solutions (will need human review)
REFERENCE_SOLUTIONS = {
    0: \"\"\"
    # Your reference solution for test question 0 goes here
    # Include algorithm, runtime analysis, and proof
    # Or leave as empty string if you don't have a solution
    \"\"\",

    # Add more as needed...
    # 1: "",  # No solution - will be flagged for human review
    # 2: "\"\"\"Your reference solution for question 2\"\"\",
}

# Load existing test data
project_root = Path.cwd()
test_input_path = project_root / "data" / "processed" / "test.jsonl"
test_output_path = project_root / "data" / "processed" / "test_with_solutions.jsonl"

if not test_input_path.exists():
    print(f"Error: {test_input_path} not found")
    exit(1)

# Load questions
questions = []
with open(test_input_path, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line:
            questions.append(json.loads(line))

print(f"Loaded {len(questions)} test questions")

# Add solutions where available
items_with_solutions = 0
items_without_solutions = 0

output_items = []
for idx, item in enumerate(questions):
    solution = REFERENCE_SOLUTIONS.get(idx, "").strip()

    if solution:
        item["solution"] = solution
        items_with_solutions += 1
    else:
        # Don't add solution field - will trigger human review
        items_without_solutions += 1

    output_items.append(item)

# Save updated test data
with open(test_output_path, 'w', encoding='utf-8') as f:
    for item in output_items:
        f.write(json.dumps(item, ensure_ascii=False) + '\\n')

print(f"\\nSaved to: {test_output_path}")
print(f"  {items_with_solutions} items with reference solutions")
print(f"  {items_without_solutions} items without solutions (will need human review)")
"""

# Write the script temporarily
with open('temp_add_solutions.py', 'w') as f:
    f.write(reference_solutions_script)

print("✓ Script created. Edit REFERENCE_SOLUTIONS in the cell above, then run:")
print("  !python temp_add_solutions.py")
print("\\nOr skip this cell if you don't have reference solutions (all items will need human review)")

## Hybrid Evaluation: Automated Metrics + Human Review

After training, evaluate your model using a hybrid approach:
- **Automated metrics** (BLEU, exact match) for questions where you have reference solutions
- **Quality checks** for all generated solutions
- **Human review** for questions without reference solutions

This section will:
1. Add your reference solutions to the test data
2. Run evaluation (automated + quality checks)
3. Generate results in JSON and CSV formats for review